In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import time 

# Timing Variance Test 

Read Values from the Time Tagger for as long as possible 
- Sample sizes will vary by 1-2 samples, we will make a uniform number in the for loop
- Max run time for the axi interface is 2**29/300e6 =~ 1.7895 seconds
- Each run we will have to manually input the period of the pulse and the slew rate 
- Also each run two sets of data: 16384 samples of waveform and all timing data

In [2]:
def get_pp_jitter(time_tagger, clk_freq, freq, threshold):
    duration = (1/freq) * (2**time_tagger.FIFO_W)

    # Set up time tagger for axi communication
    time_tagger.configure_axi(True)
    time_tagger.set_threshold(threshold)

    # Run Time Tagger
    time_tagger.send_pulsed_cmd('arm')
    time.sleep(duration)
    time_tagger.send_pulsed_cmd('disarm')

    #Iterate through samples and put into array
    sample_i = 0

    # First read call to get status
    time_tagger.send_pulsed_cmd('readout')
    first_edge = time_tagger.read_edge(sample_i)


    err, total_samples = time_tagger.read_status()

    trig_times = np.zeros(total_samples+1)
    trig_times[sample_i] = first_edge

    sample_i = sample_i + 1 
    samples_left = total_samples
    while samples_left > 0:
        time_tagger.send_pulsed_cmd('readout')
        trig_times[sample_i] = time_tagger.read_edge(sample_i%4)
        err, samples_left = time_tagger.read_status()
        if (err) :
            print('Tried to read but no samples')
            break
    
    # Now Compute all the periods 
    periods = np.array([trig_times[i+1] - trig_times[i] for i in range(total_samples-2)]) # In adc_clk cycles

    # Use adc path clk to determine jitter values
    act_period = (1/freq) * clk_freq # in adc clk cycles

    jitter = periods - act_period

    return jitter   




In [ ]:
def noise_analysis(system, clk_freq, freq, ): #Can't automate path information regarding time tagger acquistion path yet
    